# Game Theory - Cournot competition

In this project we will investigate in the Cournot competition model to see how firms compete on quantitiies. With an numerical approach, the main objective of the project is to find optimal values given the firms structure and the demand. The baseline model will include a simple demand function and a cost function for the firms which will later be used to look for the best responses. The extended model will look into how a collusion will occur when the two firms in a duopoly market end up with collaborating on quantities.

In [1]:
from scipy import optimize,arange #importing optimize and arange
from numpy import array #importing array
import matplotlib.pyplot as plt #importing plt for plotting in matplotlib
import seaborn as sns
from types import SimpleNamespace

# autoreload modules when code is run
%load_ext autoreload
%autoreload 2

%matplotlib inline

## Demand and cost function
We start by defining the demand function for firm 1 and firm 2 as follows:

$$\begin{equation} p_i(x_i, x_j) = 120-(x_i + x_j) \end{equation}$$

,where $x_i$ is the product produced by firm 1 and $x_j$ is the product produced by firm 2. The products are homogenous. 

The cost function is defined as:

$$\begin{equation} c(x) = cx \end{equation}$$

,where $x$ is the cost for firm 1 or firm 2 since they are symmetrical. We assume constant marginal costs.

In [2]:
def demand(x1,x2):
    return 120-(x1+x2)

def cost(x):
    return 30-x

## Profits function
The profits for firm 1 and firm 2 are given as:
$$\begin{equation} \pi(x_i, x_j)= p_i(x_i,x_j)x_i - c(x_i) \end{equation}$$
, if we insert the demand and cost function:
$$\begin{equation}  \pi(x_i, x_j) = 1-x_i - x_j - cx  \end{equation}$$

In [3]:
def profit(x1,x2):
    return demand(x1,x2)*x1-cost(x1)

## Reaction functions

If firm $i$ chooses the output $x_i$ its profits will be
$$\begin{equation} \pi(x_i)= x_i(120-x_i-x_j) - 30x_i \end{equation}$$


In [8]:
def reaction(x2):
    x1 = (90-x2)/2
    return x1

In [9]:
def vector_reaction(x):
    return array(x)-array([reaction(x[1]),reaction(x[0])])